In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # !pip install allennlp allennlp-models datasets nltk pandas transformers[sentencepiece] spacy

In [ ]:
# !python -m spacy download en_core_web_sm

In [1]:
# !pip install dask[complete]

In [1]:
import sentencepiece as spm 

In [2]:
import nltk

In [3]:
import pickle

In [4]:
import datasets

In [5]:
from datasets import Dataset

In [6]:
import re
import string

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [8]:
from allennlp.predictors.predictor import Predictor

In [9]:
import allennlp_models.tagging


In [10]:
import pandas as pd

In [11]:
import dask.dataframe as dd
from dask.multiprocessing import get
from dask.diagnostics import ProgressBar


In [12]:
import torch


In [13]:
device = 0 if torch.cuda.is_available() else -1

In [14]:
billsum = datasets.load_dataset("billsum")

Found cached dataset parquet (/home/bvangala1/.cache/huggingface/datasets/parquet/billsum-35284aeeba767e98/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
datasets.DatasetInfo(billsum)

DatasetInfo(description=DatasetDict({
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
}), citation='', homepage='', license='', features=None, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [ ]:
billsum

In [75]:
len(billsum['train']['text'])

18949

In [13]:
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    pattern = r'\.{19}|[\(\)\{\}\[\]\--]|`' 

    # Replace matched characters with an empty string \.{19}|
    text = re.sub(pattern, '', text)
    # Remove extra spaces and undesired elements
    text = re.sub('\s+', ' ', text).strip()  # Removes extra spaces, tabs, and newlines
    return text

In [13]:

# ##if required
# def preprocess_text(text):
#     # Lowercase
#     text = text.lower()
#     pattern = r'[\(\)\{\}\[\]\--]'
#     # Replace matched characters with an empty string
#     text = re.sub(pattern, '', text)
#     # Remove extra spaces and undesired elements
#     text = re.sub('\s+', ' ', text).strip()  # Removes extra spaces, tabs, and newlines
#     return text

# # Function to apply preprocessing to a partition of the DataFrame
# def apply_preprocessing(partition):
#     partition['processed_text'] = partition['document_text'].map(preprocess_text)
#     return partition

# # Load your dataset into a Dask DataFrame
# # Replace 'your_dataset.csv' with the path to your dataset file
# # Adjust the 'blocksize' parameter according to your memory capacity
# dask_df = dd.read_csv('your_dataset.csv', blocksize=25e6)  # 25MB chunks

# # Apply preprocessing in parallel
# processed_dask_df = dask_df.map_partitions(apply_preprocessing)

# # Optional: Convert back to a Pandas DataFrame if the dataset fits into memory
# # or if you need to perform operations that are easier or more efficient in Pandas.
# # This step can consume a lot of memory for large datasets, so it might be better
# # to keep working with the dataset in Dask or to save the processed data to disk.
# with ProgressBar():
#     processed_df = processed_dask_df.compute()

# # Saving the processed dataset to a new CSV file
# processed_df.to_csv('processed_dataset.csv', index=False)


In [14]:
def preprocess_train(ex):
    # Assuming 'text' is the field you want to preprocess
    # You can add more complex preprocessing steps here
    ex['sentences'] = [nltk.tokenize.sent_tokenize(preprocess_text(text)) for text in ex['text']]  
    # ex['processed_text'] = processed_text
    # ex[]= [nltk.tokenize.sent_tokenize(text) for text in examples['processed_text']]
    ex['max_sent_len']=[len(max(text, key=len)) for text in ex ['sentences']]
    ex['max_sent']=[max(text, key=len) for text in ex ['sentences']]
    return ex
processed_train_dataset = billsum['train'].map(preprocess_train, batched=True) #sent col by col


Loading cached processed dataset at /home/bvangala1/.cache/huggingface/datasets/parquet/billsum-35284aeeba767e98/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d0b10a1c4c862bac.arrow


In [25]:
# processed_train_dataset[0]['processed_text']

In [17]:
processed_train_dataset

Dataset({
    features: ['text', 'summary', 'title', 'sentences', 'max_sent_len', 'max_sent'],
    num_rows: 18949
})

In [23]:
# processed_train_dataset[12547]['max_sent']

In [15]:
sorted_dataset = processed_train_dataset.sort('max_sent_len', reverse=True)

Loading cached sorted indices for dataset at /home/bvangala1/.cache/huggingface/datasets/parquet/billsum-35284aeeba767e98/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-fdf31d5b634cc8cf.arrow


In [72]:
sorted_dataset[500]['max_sent_len']

2831

In [16]:
sorted_dataset[700]['max_sent']

'2 concurrent reviews.each federal and state agency a that may consider an application for a federal authorization required with respect to an application for authorization under section 3 of the natural gas act or a certificate of public convenience and necessity under section 7 of such act shall formulate and implement a plan for administrative, policy, and procedural mechanisms to enable the agency to ensure completion of federal authorizations in compliance with schedules established by the commission under section 15c1 of such act; and b in considering an aspect of an application for a federal authorization required with respect to an application for authorization under section 3 of the natural gas act or a certificate of public convenience and necessity under section 7 of such act, shall i formulate and implement a plan to enable the agency to comply with the schedule established by the commission under section 15c1 of such act; ii carry out the obligations of that agency under a

In [17]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz", cuda_device=device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [28]:
tokenizer.model_max_length

512

In [30]:
tokenizer.num_special_tokens_to_add()

2

In [31]:
tokenizer.max_len_single_sentence

510

In [34]:
# processed_train_dataset = processed_train_dataset.map(
#     lambda examples:{'sentences': [nltk.tokenize.sent_tokenize(text) for text in examples['processed_text']]},batched=True)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

In [40]:
# # longest_sentence = max(sentences, key=len)
# processed_train_dataset = processed_train_dataset.map(
#      lambda examples:{'max_sent_len': [len(max(text, key=len)) for text in examples['sentences']]},batched=True)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

In [135]:
processed_train_dataset[7]['max_sent_len']

2379

In [39]:
max(processed_train_dataset[7]['sentences'],key=len)

'a in general.notwithstanding any other provision of law, and subject to the terms and conditions of the option agreement 1 if bsl offers title acceptable to the secretary to the bsl land a the secretary shall accept a warranty deed to the bsl land and a quit claim deed to agreed to mineral interests in the bsl land; b the secretary shall convey to bsl, subject to valid existing rights and to other terms, conditions, reservations, and exceptions as may be agreed to by the secretary and bsl, fee title to the national forest system land; and c the secretary of the interior shall convey to bsl, by patent or otherwise, subject to valid existing rights and other terms, conditions, reservations, and exceptions as may be agreed to by the secretary of the interior and bsl, fee title to the blm land; 2 if bsl places title in escrow acceptable to the secretary to 11\\1/2\\ sections of the bsl land in the taylor fork area as set forth in the option agreement a the secretary shall place federal la

In [80]:
processed_train_dataset[12507]['text']

"SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Five-Star Generals Commemorative \nCoin Act''.\n\nSEC. 2. FINDINGS.\n\n    Congress finds that--\n            (1) the United States Army Command and General Staff \n        College at Fort Leavenworth, Kansas (in this Act referred to as \n        the ``CGSC'') has, in its many evolutionary forms, served this \n        Nation consistently and well for 127 years, since its founding \n        in 1881;\n            (2) the CGSC has played a decisive role in the education \n        and training of officers, particularly in their field grade \n        years of service, in times of war and peace, since its \n        establishment;\n            (3) the CGSC has had a salutatory effect on many fields of \n        battle by providing its officer student bodies the necessary \n        skills of battle management, leadership development, and the \n        most modern and effective command and staff action procedures, \n        all of w

In [64]:
df=pd.DataFrame(processed_train_dataset)

In [55]:
df.shape

(18949, 6)

In [65]:
df['max_sent_len'].max()

8586

In [30]:
df['max_sent_len'].idxmax()

14373

In [92]:
(max(df2.iloc[600]['sentences'],key=len))

'a in general.the director, in consultation with the chesapeake executive council, the chief executive of each chesapeake bay state, and the chesapeake bay commission, shall submit to congress a financial report containing 1 an interagency crosscut budget that displays, as applicable a the proposed funding for any federal restoration activity to be carried out in the succeeding fiscal year, including any planned interagency or intraagency transfer, for each of the federal agencies that carry out restoration activities; b to the extent that information is available, the estimated funding for any state restoration activity to be carried out in the succeeding fiscal year; c all expenditures for federal restoration activities from the preceding 2 fiscal years, the current fiscal year, and the succeeding fiscal year; d all expenditures, to the extent that information is available, for state restoration activities during the equivalent time period described in subparagraph c; and e a section

In [66]:
df2=df.sort_values(by='max_sent_len', ascending=False)

In [67]:
df2 = df2.reset_index(drop=True)

In [186]:
df2.iloc[1100]

text            SECTION 1. SHORT TITLE.\n\n    This Act may be...
summary         (Sec. 6) Amends the Securities Exchange Act of...
title                       Microcap Fraud Prevention Act of 1999
sentences       [section 1. short title., this act may be cite...
max_sent_len                                                 2296
Name: 1100, dtype: object

In [150]:
df2

,text,summary,title,sentences,max_sent_len
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,Five-Star Generals Commemorative Coin Act - Di...,A bill to require the Secretary of the Treasur...,"[section 1. short title., this act may be cite...",8602
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Five-Star Generals Commemorative Coin Act - Re...,A bill to require the Secretary of the Treasur...,"[section 1. short title., this act may be cite...",8602
2,SECTION 1. SHORT TITLE.\n This Act may be c...,Bankruptcy Technical Corrections Act of 2010 -...,To amend title 11 of the United States Code to...,"[section 1. short title., this act may be cite...",7941
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,Enhanced Border Security Act - Directs the Sec...,To apply counterinsurgency tactics under a coo...,"[section 1. short title., this act may be cite...",6654
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,District of Columbia-Maryland Reunion Act - Ce...,To provide for the retrocession of the Distric...,"[section 1. short title., this act may be cite...",6132
...,...,...,...,...,...
18944,SECTION 1. SHORT TITLE.\n\n This Act may be...,ATF Enforcement Act This bill amends the feder...,ATF Enforcement Act,"[section 1. short title., this act may be cite...",234
18945,SECTION 1. PURPOSE.\n The purpose of this A...,Makes technical corrections to provisions of T...,To make technical corrections to the United St...,"[section 1. purpose., the purpose of this act ...",222
18946,SECTION 1. SHORT TITLE.\n\n This Act may be...,Priorities in Education Spending Act - Repeals...,To repeal ineffective or unneccesary education...,"[section 1. short title., this act may be cite...",194
18947,SECTION 1. TABLE OF CONTENTS.\n\nSec. 1. Table...,Prohibits the use of federal funding for speci...,To provide that Federal funds may not be used ...,"[section 1. table of contents., sec., 1. table...",184


In [103]:
df.iloc[12508]

text            SECTION 1. SHORT TITLE.\n\n    This Act may be...
summary         Federal Payment Reauthorization Act of 1994 - ...
title                 Federal Payment Reauthorization Act of 1994
sentences       [section 1. short title., this act may be cite...
max_sent_len                                                  699
Name: 12508, dtype: object

In [133]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sample:
    count += 1
    combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

    if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
        chunk += sentence + " " # add the sentence to the chunk
        length = combined_length # update the length counter

        # if it is the last sentence
        if count == len(sentences) - 1:
              chunks.append(chunk.strip()) # save the chunk
    
    else: 
        chunks.append(chunk.strip()) # save the chunk
    
        # reset 
        length = 0 
        chunk = ""

        # take care of the overflow sentence
        chunk += sentence + " "
        length = len(tokenizer.tokenize(sentence))
len(chunks)

Token indices sequence length is longer than the specified maximum sequence length for this model (1042 > 512). Running this sequence through the model will result in indexing errors


1

In [82]:
s='''2 concurrent reviews.each federal and state agency a that may consider an application for a federal authorization required with respect to an application for authorization under section 3 of the natural gas act or a certificate of public convenience and necessity under section 7 of such act shall formulate and implement a plan for administrative, policy, and procedural mechanisms to enable the agency to ensure completion of federal authorizations in compliance with schedules established by the commission under section 15c1 of such act; and b in considering an aspect of an application for a federal authorization required with respect to an application for authorization under section 3 of the natural gas act or a certificate of public convenience and necessity under section 7 of such act, shall i formulate and implement a plan to enable the agency to comply with the schedule established by the commission under section 15c1 of such act; ii carry out the obligations of that agency under applicable law concurrently, and in conjunction with, the projectrelated nepa review conducted by the commission, and in compliance with the schedule established by the commission under section 15c1 of such act, unless the agency notifies the commission in writing that doing so would impair the ability of the agency to conduct needed analysis or otherwise carry out such obligations; iii transmit to the commission a statement i acknowledging receipt of the schedule established by the commission under section 15c1 of the natural gas act; and ii setting forth the plan formulated under clause i of this subparagraph; iv not later than 30 days after the agency receives such application for a federal authorization, transmit to the applicant a notice i indicating whether such application is ready for processing; and ii if such application is not ready for processing, that includes a comprehensive description of the information needed for the agency to determine that the application is ready for processing; v determine that such application for a federal authorization is ready for processing for purposes of clause iv if such application is sufficiently complete for the purposes of commencing consideration, regardless of whether supplemental information is necessary to enable the agency to complete the consideration required by law with respect to such application; and vi not less often than once every 90 days, transmit to the commission a report describing the progress made in considering such application for a federal authorization.'''

In [22]:
sample = '''a in general.notwithstanding any other provision of law, and subject to the terms and conditions of the option agreement 1 if bsl offers title acceptable to the secretary to the bsl land a the secretary shall accept a warranty deed to the bsl land and a quit claim deed to agreed to mineral interests in the bsl land; b the secretary shall convey to bsl, subject to valid existing rights and to other terms, conditions, reservations, and exceptions as may be agreed to by the secretary and bsl, fee title to the national forest system land; and c the secretary of the interior shall convey to bsl, by patent or otherwise, subject to valid existing rights and other terms, conditions, reservations, and exceptions as may be agreed to by the secretary of the interior and bsl, fee title to the blm land; 2 if bsl places title in escrow acceptable to the secretary to 11\\1/2\\ sections of the bsl land in the taylor fork area as set forth in the option agreement a the secretary shall place federal land in the bangtail and doe creek areas of the gallatin national forest, as identified in the option agreement, in escrow pending conveyance to the secretary of the taylor fork land, as identified in the option agreement in escrow; b the secretary, subject to the availability of funds, shall purchase 7\\1/2\\ sections of bsl land in the taylor fork area held in escrow and identified in the option agreement at a purchase price of $4,150,000; and c the secretary shall acquire the 4 taylor fork sections identified in the option agreement remaining in escrow, and any of the 6 sections referred to in subparagraph b for which funds are not available, by providing bsl with timber sale receipts from timber sales on the gallatin national forest and other eastside national forests in the state of montana in accordance with subsection c; and 3a as funds or timber sale receipts are received by bsl i the deeds to an equivalent value of bsl taylor fork land held in escrow shall be released and conveyed to the secretary; and ii the escrow of deeds to an equivalent value of federal land shall be released to the secretary in accordance with the terms of the option agreement; or b if funds or timber sale receipts are not provided to bsl as provided in the option agreement, bsl shall be entitled to receive patents and deeds to an equivalent value of the federal land held in escrow.'''

In [18]:
indices_to_keep = range(600, len(sorted_dataset))

# Select the rows to keep, effectively removing the first 600 rows
reduced_dataset = sorted_dataset.select(indices_to_keep)
print("Original dataset size:", len(sorted_dataset))
print("Reduced dataset size:", len(reduced_dataset))

Original dataset size: 18949
Reduced dataset size: 18349


In [89]:
sorted_dataset[600]['max_sent_len']

2663

In [87]:
reduced_dataset[0]['max_sent_len']

2663

In [57]:
# Assuming dataset is a Hugging Face Dataset with 18,000 records
num_records = len(reduced_dataset)
batch_size = 1000
dataset_batch = {}
for start_idx in range(0, num_records, batch_size):
    end_idx = min(start_idx + batch_size, num_records)  # Ensure not to exceed the dataset size
    batch = reduced_dataset.select(range(start_idx, end_idx)).to_dict()  # Selecting the batch
    dataset_batch[f'db_{start_idx//batch_size + 1}']=batch
    #dataset.save_to_disk(f'./dataset_batch_{start_idx//batch_size + 1}')  # Saving the batch to disk


In [58]:
len(dataset_batch['db_1'])

6

In [61]:
(dataset_batch['db_1']['max_sent_len'])

[2663,
 2662,
 2659,
 2657,
 2657,
 2656,
 2655,
 2653,
 2652,
 2652,
 2652,
 2651,
 2650,
 2650,
 2650,
 2646,
 2646,
 2645,
 2645,
 2644,
 2642,
 2641,
 2640,
 2638,
 2636,
 2636,
 2635,
 2629,
 2628,
 2628,
 2627,
 2626,
 2625,
 2624,
 2621,
 2621,
 2618,
 2618,
 2618,
 2618,
 2616,
 2615,
 2613,
 2613,
 2611,
 2610,
 2607,
 2605,
 2604,
 2604,
 2602,
 2598,
 2598,
 2597,
 2595,
 2594,
 2593,
 2593,
 2592,
 2591,
 2591,
 2588,
 2588,
 2587,
 2586,
 2584,
 2584,
 2582,
 2581,
 2580,
 2579,
 2575,
 2575,
 2573,
 2572,
 2570,
 2570,
 2570,
 2570,
 2567,
 2567,
 2565,
 2564,
 2564,
 2564,
 2564,
 2561,
 2561,
 2560,
 2559,
 2555,
 2555,
 2554,
 2553,
 2553,
 2553,
 2550,
 2550,
 2550,
 2549,
 2548,
 2547,
 2546,
 2546,
 2546,
 2546,
 2545,
 2544,
 2543,
 2542,
 2542,
 2542,
 2542,
 2536,
 2536,
 2535,
 2534,
 2534,
 2531,
 2530,
 2530,
 2529,
 2529,
 2529,
 2529,
 2529,
 2528,
 2528,
 2528,
 2527,
 2526,
 2524,
 2524,
 2524,
 2524,
 2523,
 2520,
 2520,
 2520,
 2520,
 2519,
 2519,
 2519,

In [34]:
(dataset_batch['db_19'][0].keys())

dict_keys(['text', 'summary', 'title', 'sentences', 'max_sent_len', 'max_sent'])

In [28]:
def predict_srl(sents):
    srl_list = []
    for ix,s1 in enumerate(sents['sentences']):
        # print(s1)
        sub_srl_list = []
        for s2 in s1:
            # print(s2)
            try:
                 sub_srl_list.append(predictor.predict(s2))
            except Exception as e:
                print(f'{ix}',end ='|')
                sub_srl_list.append('e')
        # print('-------------')
        # print(sub_srl_list)
        # print('-------------')
        # print(f'dine: {ix}')
        srl_list.append(sub_srl_list)
    sents['srl_sentences'] = srl_list
    return sents

In [47]:
srl_dataset={}

In [222]:
import time

In [228]:
for indx in range(6,0,-1):
    start_time = time.time()
    dataset_batch[f'db_{indx}']['db_srl']=[]
    for i,data in enumerate(dataset_batch[f'db_{indx}']['sentences']):
        print(f'{i}:B',end = '|')
        temp = []
        for j,sentence in enumerate(data):
            try:
                temp.append(predictor.predict(sentence))
            except Exception as e:
                print(f'[{i},{j}]',end='|')
                temp.append('e')
        dataset_batch[f'db_{indx}']['db_srl'].append(temp)
        print(f'{i}:D',end='|')
    with open(f'db_srl_{indx}.pkl', 'wb') as f:
        pickle.dump(dataset_batch[f'db_{indx}'], f)
    time.sleep(3)
    end_time = time.time()  
    iteration_time = (end_time - start_time )/60
    print(f"Iteration {indx} took {iteration_time:.2f} minutes")
    print(f'=========================================={indx}-END================================================')

0:B|0:D|1:B|1:D|2:B|2:D|3:B|3:D|4:B|4:D|5:B|5:D|6:B|6:D|7:B|7:D|8:B|8:D|9:B|9:D|10:B|10:D|11:B|11:D|12:B|12:D|13:B|13:D|14:B|14:D|15:B|15:D|16:B|16:D|17:B|17:D|18:B|18:D|19:B|19:D|20:B|20:D|21:B|21:D|22:B|22:D|23:B|23:D|24:B|24:D|25:B|25:D|26:B|26:D|27:B|27:D|28:B|28:D|29:B|29:D|30:B|30:D|31:B|31:D|32:B|32:D|33:B|33:D|34:B|34:D|35:B|35:D|36:B|36:D|37:B|37:D|38:B|38:D|39:B|39:D|40:B|40:D|41:B|41:D|42:B|42:D|43:B|43:D|44:B|44:D|45:B|45:D|46:B|46:D|47:B|47:D|48:B|48:D|49:B|49:D|50:B|50:D|51:B|51:D|52:B|52:D|53:B|53:D|54:B|54:D|55:B|55:D|56:B|56:D|57:B|57:D|58:B|58:D|59:B|59:D|60:B|60:D|61:B|61:D|62:B|62:D|63:B|63:D|64:B|64:D|65:B|65:D|66:B|66:D|67:B|67:D|68:B|68:D|69:B|69:D|70:B|70:D|71:B|71:D|72:B|72:D|73:B|73:D|74:B|74:D|75:B|75:D|76:B|76:D|77:B|77:D|78:B|78:D|79:B|79:D|80:B|80:D|81:B|81:D|82:B|82:D|83:B|83:D|84:B|84:D|85:B|85:D|86:B|86:D|87:B|87:D|88:B|88:D|89:B|89:D|90:B|90:D|91:B|91:D|92:B|92:D|93:B|93:D|94:B|94:D|95:B|95:D|96:B|96:D|97:B|97:D|98:B|98:D|99:B|99:D|100:B|100:D|101:B|10

In [216]:
dataset_batch['db_7']['db_srl']=[]

In [217]:
%%time
for i,data in enumerate(dataset_batch['db_7']['sentences']):
    print(f'{i}:B',end = '|')
    temp = []
    for j,sentence in enumerate(data):
        try:
            temp.append(predictor.predict(sentence))
        except Exception as e:
            print(f'[{i},{j}]',end='|')
            temp.append('e')
    dataset_batch['db_7']['db_srl'].append(temp)
    print(f'{i}:D',end='|')
with open('db_srl_7.pkl', 'wb') as f:
    pickle.dump(dataset_batch['db_7'], f)

0:B|0:D|1:B|1:D|2:B|2:D|3:B|3:D|4:B|4:D|5:B|5:D|6:B|6:D|7:B|7:D|8:B|8:D|9:B|9:D|10:B|10:D|11:B|11:D|12:B|12:D|13:B|13:D|14:B|14:D|15:B|15:D|16:B|16:D|17:B|17:D|18:B|18:D|19:B|19:D|20:B|20:D|21:B|21:D|22:B|22:D|23:B|23:D|24:B|24:D|25:B|25:D|26:B|26:D|27:B|27:D|28:B|28:D|29:B|29:D|30:B|30:D|31:B|31:D|32:B|32:D|33:B|33:D|34:B|34:D|35:B|35:D|36:B|36:D|37:B|37:D|38:B|38:D|39:B|39:D|40:B|40:D|41:B|41:D|42:B|42:D|43:B|43:D|44:B|44:D|45:B|45:D|46:B|46:D|47:B|47:D|48:B|48:D|49:B|49:D|50:B|50:D|51:B|51:D|52:B|52:D|53:B|53:D|54:B|54:D|55:B|55:D|56:B|56:D|57:B|57:D|58:B|58:D|59:B|59:D|60:B|60:D|61:B|61:D|62:B|62:D|63:B|63:D|64:B|64:D|65:B|65:D|66:B|66:D|67:B|67:D|68:B|68:D|69:B|69:D|70:B|70:D|71:B|71:D|72:B|72:D|73:B|73:D|74:B|74:D|75:B|75:D|76:B|76:D|77:B|77:D|78:B|78:D|79:B|79:D|80:B|80:D|81:B|81:D|82:B|82:D|83:B|83:D|84:B|84:D|85:B|85:D|86:B|86:D|87:B|87:D|88:B|88:D|89:B|89:D|90:B|90:D|91:B|91:D|92:B|92:D|93:B|93:D|94:B|94:D|95:B|95:D|96:B|96:D|97:B|97:D|98:B|98:D|99:B|99:D|100:B|100:D|101:B|10

In [232]:
len(dataset_batch['db_1']['db_srl'][8])

42

In [231]:
len(dataset_batch['db_1']['sentences'][8])

42

In [235]:
dataset_batch['db_1'].keys()

dict_keys(['text', 'summary', 'title', 'sentences', 'max_sent_len', 'max_sent', 'db_srl'])

In [132]:
print(dataset_batch['db_1']['db_srl'][666][0])

KeyError: 'db_srl'

In [195]:
dataset_batch['db_10'].keys()

dict_keys(['text', 'summary', 'title', 'sentences', 'max_sent_len', 'max_sent'])

In [236]:
with open('db_srl_all.pkl', 'wb') as f:
    pickle.dump(dataset_batch, f)

In [29]:
for idx in range(1,20):
    print(f'----ENTER BATCH {idx} ----')
    for v in dataset_batch[f'db_{idx}']:
    try:
        # srl_dataset[f'db_srl_{idx}']= dataset_batch[f'db_{idx}'].map(predict_srl, batched = True)
        srl_dataset[f'db_srl_{idx}']
    except Exception as e:
        print(f'{idx} :: e')
        srl_dataset[f'db_srl_{idx}'] = 'e'
    srl_dataset[f'db_srl_{idx}'].save_to_disk(f'db_srl_{idx}')
    print(f'----EXIT BATCH {idx} ----')


----ENTER BATCH 1 ----


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

2|4|8|14|15|16|17|29|30|31|34|35|36|38|40|46|58|63|82|87|102|105|106|112|118|119|136|137|139|157|168|177|180|188|189|190|193|223|228|249|255|308|326|332|334|361|371|379|433|435|436|447|451|453|456|478|490|491|513|513|535|543|559|563|611|642|643|660|695|705|743|754|798|815|847|848|1 :: e


KeyError: 'db_1'

In [30]:
srl_dataset

{'db_srl_1': 'e'}

In [118]:
srl_dataset=reduced_dataset.map( predict_srl, batched = True)

Map:   0%|          | 0/18349 [00:00<?, ? examples/s]

2:|4:|8:|14:|15:|16:|17:|29:|30:|31:|34:|35:|36:|38:|40:|46:|58:|63:|82:|87:|102:|105:|106:|112:|118:|119:|136:|137:|139:|157:|168:|177:|180:|188:|189:|190:|193:|223:|228:|249:|255:|308:|326:|332:|334:|361:|371:|379:|433:|435:|436:|447:|451:|453:|456:|478:|490:|491:|513:|513:|535:|543:|559:|563:|611:|642:|643:|660:|695:|705:|743:|754:|798:|815:|847:|848:|

ArrowInvalid: cannot mix struct and non-struct, non-null values

In [106]:
srl_dataset[1]['srl_sentences'][0]

{'verbs': [], 'words': ['text']}

In [ ]:
# predictions = [predictor.predict(s)]

In [17]:
len(prediction)

2

In [193]:
prediction

{'verbs': [{'verb': 'finds',
   'description': '[ARG0: a findings.congress] [V: finds] [ARG1: that 1 the four corners monument is nationally significant as the only geographic location in the united states where 4 state boundaries meet ; 2 the states with boundaries that meet at the four corners area are arizona , colorado , new mexico , and utah ; 3 between 1868 and 1875 the boundary lines that created the four corners were drawn , and in 1899 a monument was erected at the site ; 4 a united states postal stamp will be issued in 1999 to commemorate the centennial of the original boundary marker ; 5 the four corners area is distinct in character and possesses important historical , cultural , and prehistoric values and resources within the surrounding cultural landscape ; 6 although there are no permanent facilities or utilities at the four corners monument tribal park , each year the park attracts approximately 250,000 visitors ; 7 the area of the four corners monument tribal park fall

In [37]:
# len(prediction[0]['verbs'])

In [60]:
prediction[0]['verbs'][1]

{'verb': 'offers',
 'description': 'a in general.notwithstanding any other provision of law , and subject to the terms and conditions of the option agreement 1 if [ARG0: bsl] [V: offers] [ARG1: title acceptable to the secretary to the bsl land a] the secretary shall accept a warranty deed to the bsl land and a quit claim deed to agreed to mineral interests in the bsl land ; b the secretary shall convey to bsl , subject to valid existing rights and to other terms , conditions , reservations , and exceptions as may be agreed to by the secretary and bsl , fee title to the national forest system land ; and c the secretary of the interior shall convey to bsl , by patent or otherwise , subject to valid existing rights and other terms , conditions , reservations , and exceptions as may be agreed to by the secretary of the interior and bsl , fee title to the blm land ; 2 if bsl places title in escrow acceptable to the secretary to 11\\1/2\\ sections of the bsl land in the taylor fork area as s

In [84]:
len(prediction['verbs'])

55

In [59]:
prediction['verbs'][1]

{'verb': 'be',
 'description': 'b matters to [V: be] addressed.in the review and report under subsection a , the commission shall 1 examine changes in law and management practices and policies that should be adopted to a reduce administration and management overhead costs by 50 percent and the overall costs of land management by a minimum of 30 percent ; b devote a greater proportion of the land management agencies resources to the provision of services in the field and a lesser proportion to procedural and administrative matters ; c improve service to land and resource users ; d simplify land use planning requirements , including the provision that any challenge to a land use plan be made in the context of a challenge to a specific action proposed to be taken under the plan and not as a challenge to the plan on its face ; e simplify the administrative appeals process , providing for no more than 1 level of intermediate appeal of an initial land management agencys decision before appea

In [94]:
print(f"PyTorch version: {torch.__version__}")

# Print the CUDA version that PyTorch is built with
print(f"CUDA version for PyTorch: {torch.version.cuda}")

PyTorch version: 1.12.1+cu116
CUDA version for PyTorch: 11.6


In [208]:
# pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116


In [16]:
with open('db_srl_all.pkl', 'rb') as f:
    srlDB = pickle.load(f)

In [18]:
srlDB.keys()

dict_keys(['db_1', 'db_2', 'db_3', 'db_4', 'db_5', 'db_6', 'db_7', 'db_8', 'db_9', 'db_10', 'db_11', 'db_12', 'db_13', 'db_14', 'db_15', 'db_16', 'db_17', 'db_18', 'db_19'])

In [25]:
(srlDB['db_19']['db_srl'])

349

In [26]:
sample = {key: value[0] for key, value in srlDB['db_19'].items()}

In [28]:
sample.keys()

dict_keys(['text', 'summary', 'title', 'sentences', 'max_sent_len', 'max_sent', 'db_srl'])

In [29]:
sample['db_srl']

[{'verbs': [], 'words': ['section', '1', '.', 'short', 'title', '.']},
 {'verbs': [{'verb': 'may',
    'description': "this act [V: may] be cited as the broadband adoption act of 2013 '' .",
    'tags': ['O',
     'O',
     'B-V',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O']},
   {'verb': 'be',
    'description': "this act may [V: be] cited as the broadband adoption act of 2013 '' .",
    'tags': ['O',
     'O',
     'O',
     'B-V',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O']},
   {'verb': 'cited',
    'description': "[ARG1: this act] [ARGM-MOD: may] be [V: cited] [ARG2: as the broadband adoption act of 2013] '' .",
    'tags': ['B-ARG1',
     'I-ARG1',
     'B-ARGM-MOD',
     'O',
     'B-V',
     'B-ARG2',
     'I-ARG2',
     'I-ARG2',
     'I-ARG2',
     'I-ARG2',
     'I-ARG2',
     'I-ARG2',
     'O',
     'O']}],
  'words': ['this',
   'act',
   'may',
   'be'

In [ ]:
'''
        {"words": [...],
         "verbs": [
            {"verb": "...", "description": "...", "tags": [...]},
            ...
            {"verb": "...", "description": "...", "tags": [...]},
        ]}
'''